<a href="https://colab.research.google.com/github/subro608/linear-regression-single-variable/blob/master/spellchecker_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


In [0]:
import numpy as np


right = []
wrong = []
word2dict = {}


with open('spell-errors.txt') as f:
  for line in f:
    right_words = line.split(':')
    right.append(right_words[0])
    print(right_words[0])
    wrong_words = right_words[1].split(',')
   
    wrong.append(wrong_words)
    word2dict[right_words[0]]  = wrong_words

alphabet = 'abcdefghijklmnopqrstuvwxyz -'

def split(word): 
    return [char for char in word] 

In [0]:
word_dict = {}
for i,char in enumerate(alphabet):
  word_dict[char] = i
  print(char,i)

In [0]:

def edittype(candidate,word):
  
  
  edit = [False]*4
  correct = ''
  error = ''
  x = ''
  y = ''
  for i in range(min([len(word),len(candidate)])-1):
    if candidate[0:i+1] != word[0:i+1]:
      if candidate[i:] == word[i-1:]:
        edit[1] = True
        correct = candidate[i-1]
        error = ''
        x = candidate[i-2]
        w = candidate[i-2]+candidate[i-1]
        break
      elif candidate[i:] == word[i+1:]:
        
        correct = ''
        error = word[i]
        if i == 0:
          w = '_'
          x = '_'+error
        else:
          w = word[i-1]
          x = word[i-1]+error
        edit[0] = True
        break
      if candidate[i+1:] == word[i+1:]:
        
        edit[2] = True
        error = candidate[i]
        correct = word[i]
        x = error
        w = correct
        break
      if candidate[i] == word[i+1] and candidate[i+2:] == word[i+2:]:
        
        edit[3] = True
        correct = candidate[i]+candidate[i+1]
        error = word[i]+word[i+1]
        x = error
        w = correct
        break
  candidate = candidate[::-1]
  word = word[::-1] 
  for i in range(min([len(word),len(candidate)])-1):
    
    if candidate[0:i+1] != word[0:i+1]:
      
      if candidate[i:] == word[i-1:]:
        
        edit[1] = True
        correct = candidate[i-1]
        error = ''
        x = candidate[i-2]
        w = candidate[i-2]+candidate[i-1]
        break
      elif candidate[i:] == word[i+1:]:
        
        correct = ''
        error = word[i]
        if i == 0:
          w = '_'
          x = '_'+error
        else:
          w = word[i-1]
          x = word[i-1]+error
        edit[0] = True
        break
      if candidate[i+1:] == word[i+1:]:
        
        edit[2] = True
        error = candidate[i]
        correct = word[i]
        x = error
        w = correct
        break
      if candidate[i] == word[i+1] and candidate[i+2:] == word[i+2:]:
        
        edit[3] = True
        correct = candidate[i]+candidate[i+1]
        error = word[i]+word[i+1]
        x = error
        w = correct
        break
  if word == candidate:
    return "not required",'',''
  if edit[1]:
    return "Deletion",x,w
  elif edit[0]:
    return "Insertion",x,w
  elif edit[2]:
    return "Substitution",x,w
  elif edit[3]:
    return "Reversal",x,w
  else :
    return "not required",'',''  

In [0]:
sub_matrix = np.zeros((28,28))
add_matrix = np.zeros((28,28))
del_matrix = np.zeros((28,28))
trans_matrix = np.zeros((28,28))

def subconfusion_matrix(sub_matrix,x,y):
  if x in split(alphabet) and y in split(alphabet):
    
    sub_matrix[word_dict[x]][word_dict[y]] += 1
  
  
def addconfusion_matrix(add_matrix,x,y):
  if x in split(alphabet) and y in split(alphabet):
    
    add_matrix[word_dict[x]][word_dict[y]] += 1
  
  
def delconfusion_matrix(del_matrix,x,y):
  
  if x in split(alphabet) and y in split(alphabet):
    
    del_matrix[word_dict[x]][word_dict[y]] += 1
  
  
  
def transconfusion_matrix(trans_matrix,x,y):
  if x in split(alphabet) and y in split(alphabet):
    
    trans_matrix[word_dict[x]][word_dict[y]] += 1

In [0]:

for right_word in right:
  for wrong_word in word2dict[right_word]:
    new_wrong_word = wrong_word.lstrip()
    type,x,w = edittype(new_wrong_word.strip('\n'),right_word.strip('\n'))
    
    

    
    if type == 'Deletion':
      
      
      delconfusion_matrix(del_matrix,w[0],w[1])
        
        
    elif type == 'Insertion':
      
      addconfusion_matrix(add_matrix,x[1],w)
        
        
    elif type == 'Substitution':
      
      subconfusion_matrix(sub_matrix,x,w)
        
        
    elif type == 'Reversal':
      
      transconfusion_matrix(trans_matrix,x[1],w[1])  

In [0]:
print(np.sum(del_matrix),np.sum(sub_matrix),np.sum(add_matrix),np.sum(trans_matrix))

In [0]:
def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)


In [0]:
word = 'higher'
splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
deletes    = [a + b[1:] for a, b in splits if b]
deletes

In [0]:
def known_edits1(word):
  return set(e1 for e1 in edits1(word) if e1 in right)

In [0]:
def prob_estimate(candidate_words,error_word):
  prob_word = {}

  for word in candidate_words:
    type,x,w = edittype(error_word,word)
    if type == 'Deletion':  
      if w[0] in split(alphabet) and w[1] in split(alphabet):
        
        count_num = del_matrix[word_dict[w[0]]][word_dict[w[1]]]
        total_count = del_matrix[word_dict[w[0]]][word_dict[w[1]]]+sub_matrix[word_dict[w[0]]][word_dict[w[1]]]+add_matrix[word_dict[w[0]]][word_dict[w[1]]]+trans_matrix[word_dict[w[0]]][word_dict[w[1]]]
      
      
    elif type == 'Insertion':
      if x[1] in split(alphabet) and w in split(alphabet):
          
        count_num = add_matrix[word_dict[x[1]]][word_dict[w]]
        total_count = add_matrix[word_dict[w]] + del_matrix[word_dict[w]] + sub_matrix[word_dict[w]] + trans_matrix[word_dict[w]]
      
    elif type == 'Substitution':
      if x in split(alphabet) and w in split(alphabet):
        count_num = sub_matrix[word_dict[x]][word_dict[w]] 
        total_count = sub_matrix[word_dict[x]][word_dict[w]]+del_matrix[word_dict[x]][word_dict[w]]+add_matrix[word_dict[x]][word_dict[w]]+trans_matrix[word_dict[x]][word_dict[w]] 
                             
      
        
    elif type == 'Reversal':
      if x[1] in split(alphabet) and w[1] in split(alphabet):
        count_num = trans_matrix[word_dict[x[1]]][word_dict[w[1]]]
        total_count = trans_matrix[word_dict[x[1]]][word_dict[w[1]]]+add_matrix[word_dict[x[1]]][word_dict[w[1]]]+sub_matrix[word_dict[x[1]]][word_dict[w[1]]]+del_matrix[word_dict[x[1]]][word_dict[w[1]]]
   
      
    prob_word[word] = count_num/np.sum(total_count)
  return prob_word  
    
    
    
      
      
    
    

In [0]:
prob_estimate(known_edits1('speling'),'speling')

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
word_freq = {}
total_count = []
original_words = []

In [0]:
with open('eng_news_2016_1M-words.txt') as f:
  for line in f:
    
    parts = line.split()
    
    array_parts = np.array(parts)
    if array_parts.shape[0] == 4:
      word_freq[array_parts[1]] = array_parts[3]
      total_count.append(array_parts[3])
      original_words.append(array_parts[1])   

In [0]:
sum_value = 0
size = len(total_count)
for i in range(size):
  sum_value += int(total_count[i])

In [0]:
unique_words = len(set(original_words))
print(unique_words)

for value in freq:
  total_count += value


In [0]:
def prob_unigram(word):
  try:
    
    prob = (int(word_freq[word])+1)/(sum_value+unique_words)
    return prob
   
  except:
    print('word not available')
  


In [0]:

prob_unigram('speling')

In [0]:
def spellchecker(word):
  try:
    
    
    known_words = list(known_edits1(word))
    first_word = known_words[0]
    prob_dict = prob_estimate(known_words,word)
    prob = {}
    for key_word in known_words:
      
      w = prob_dict[key_word] * prob_unigram(key_word)
      prob[key_word] = w
    k = prob[first_word]
    l = known_words[0]
    for key in known_words:
      
      if prob[key] > k:
        
        k = prob[key]
        l = key
    return l 
  except:
    print('cannot find the correct spelling')
    
  

In [0]:


spellchecker('helo')

In [0]:
 edittype('helol','hello')